In [1]:
import os
import geopandas as gpd

In [2]:
%store -r

In [3]:
connex_big = gpd.pd.read_csv(os.path.join(output_dir, 'california-building-points-to-big-hexes.csv'))

In [4]:
connex_small = gpd.pd.read_csv(os.path.join(output_dir, 'california-building-points-to-small-hexes.csv'))

In [5]:
len(connex_big), len(connex_small)

(10988525, 10988525)

In [10]:
hexes_big = gpd.read_file(os.path.join(output_dir, 'hexes-big.shp'))

In [11]:
hexes_small = gpd.read_file(os.path.join(output_dir, 'hexes-small.shp'))

In [12]:
len(hexes_big), len(hexes_small)

(6700, 17622)

In [14]:
points = gpd.pd.read_csv(os.path.join(output_dir, "buildings-points-with-analysis.csv"))

In [19]:
len(points)

10988525

In [17]:
merged_points_big = points.merge(connex_big, on="FID", how="inner")

In [18]:
merged_points_small = points.merge(connex_small, on="FID", how="inner")

In [20]:
len(merged_points_big), len(merged_points_small)

(10988525, 10988525)

In [23]:
filtered_points_big = merged_points_big[~gpd.pd.isnull(merged_points_big.hex_id)]

In [24]:
filtered_points_small = merged_points_small[~gpd.pd.isnull(merged_points_small.hex_id)]

In [25]:
filtered_points_big.hex_id = filtered_points_big.hex_id.astype(int)

/home/palewire/.virtualenvs/california-fire-zone-analysis/local/lib/python3.6/site-packages/pandas/core/generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [26]:
filtered_points_small.hex_id = filtered_points_small.hex_id.astype(int)

In [29]:
filtered_points_big['in_firezone'] = ~gpd.pd.isnull(filtered_points_big.fire_zone_type)

/home/palewire/.virtualenvs/california-fire-zone-analysis/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [28]:
filtered_points_small['in_firezone'] = ~gpd.pd.isnull(filtered_points_small.fire_zone_type)

/home/palewire/.virtualenvs/california-fire-zone-analysis/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [30]:
grouped_hexes_big = filtered_points_big.groupby([
    "hex_id",
    "in_firezone"
]).size().rename("building_count").reset_index()

In [31]:
grouped_hexes_small = filtered_points_small.groupby([
    "hex_id",
    "in_firezone"
]).size().rename("building_count").reset_index()

In [32]:
pivoted_hexes_big = grouped_hexes_big.pivot(
    index="hex_id",
    columns="in_firezone",
    values="building_count"
).reset_index().rename(columns={True: "in_zone", False: "out_zone"}).fillna(0)

In [33]:
pivoted_hexes_small = grouped_hexes_small.pivot(
    index="hex_id",
    columns="in_firezone",
    values="building_count"
).reset_index().rename(columns={True: "in_zone", False: "out_zone"}).fillna(0)

In [34]:
pivoted_hexes_big['total'] = pivoted_hexes_big.in_zone + pivoted_hexes_big.out_zone

In [35]:
pivoted_hexes_small['total'] = pivoted_hexes_small.in_zone + pivoted_hexes_small.out_zone

In [36]:
pivoted_hexes_big['in_pct'] = pivoted_hexes_big.in_zone / pivoted_hexes_big.total

In [37]:
pivoted_hexes_small['in_pct'] = pivoted_hexes_small.in_zone / pivoted_hexes_small.total

In [38]:
merged_hexes_big = hexes_big.merge(pivoted_hexes_big, on="hex_id")

In [39]:
merged_hexes_small = hexes_small.merge(pivoted_hexes_small, on="hex_id")

In [55]:
state = gpd.read_file(os.path.join(input_dir, "state", "state.shp"))[['geometry']]

In [57]:
hexes_clipped_big = gpd.overlay(merged_hexes_big, state, how='intersection')

In [58]:
hexes_clipped_small = gpd.overlay(merged_hexes_small, state, how='intersection')

In [60]:
hexes_clipped_big.head()

,hex_id,out_zone,in_zone,total,in_pct,geometry
0,6819,2.0,1.0,3.0,0.333333,"POLYGON ((-124.3443320877625 40.2603865134873,..."
1,6820,1.0,3.0,4.0,0.750000,POLYGON ((-124.3514615709291 40.35680263271846...
2,6821,11.0,0.0,11.0,0.000000,POLYGON ((-124.3883783121074 40.44340521347536...
3,6822,35.0,0.0,35.0,0.000000,POLYGON ((-124.3464317109737 40.52383090983665...
5,6935,41.0,55.0,96.0,0.572917,POLYGON ((-124.3412818462808 40.28529676365753...


In [61]:
hexes_clipped_big.to_file(os.path.join(output_dir, "big-hexes-with-analysis.shp"))

/home/palewire/.virtualenvs/california-fire-zone-analysis/local/lib/python3.6/site-packages/geopandas/io/file.py:108: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():


In [62]:
hexes_clipped_small.to_file(os.path.join(output_dir, "small-hexes-with-analysis.shp"))